# Chapter 16: Geometric Algorithms

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_16_geometric_algorithms.ipynb)

**Geometric algorithms** process and manipulate 3D meshes for modeling, optimization, and analysis. This chapter covers convex hulls, mesh simplification, subdivision surfaces, and computational geometry.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
from typing import Tuple, List, Optional, Set
from dataclasses import dataclass
import heapq

In [ ]:
class Vec3:
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x, self.y, self.z = x, y, z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar: float):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self) -> float:
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        return self / l if l > 0 else Vec3(0, 0, 0)
    
    def to_array(self):
        return np.array([self.x, self.y, self.z])

@dataclass
class Vertex:
    position: Vec3
    index: int = -1

@dataclass
class Triangle:
    v0: int
    v1: int
    v2: int

class Mesh:
    def __init__(self):
        self.vertices = []
        self.triangles = []
    
    def add_vertex(self, v: Vec3) -> int:
        idx = len(self.vertices)
        self.vertices.append(Vertex(v, idx))
        return idx
    
    def add_triangle(self, v0: int, v1: int, v2: int):
        self.triangles.append(Triangle(v0, v1, v2))

print("✓ Base classes loaded")

## 1. Convex Hull (Gift Wrapping Algorithm)

### 2D Convex Hull

The **Jarvis March** or **Gift Wrapping** algorithm:

1. Start with leftmost point
2. Find the point that makes smallest counter-clockwise angle
3. Repeat until back to start

**Time Complexity**: O(nh) where h is hull size

### Cross Product Test

For points $\mathbf{p}_1, \mathbf{p}_2, \mathbf{p}_3$:

$$
\text{turn} = (\mathbf{p}_2 - \mathbf{p}_1) \times (\mathbf{p}_3 - \mathbf{p}_1)
$$

- $\text{turn} > 0$: counter-clockwise
- $\text{turn} < 0$: clockwise
- $\text{turn} = 0$: collinear

In [ ]:
def cross_product_2d(o, a, b):
    """2D cross product: (a-o) × (b-o)"""
    return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])

def convex_hull_2d(points):
    """Gift wrapping algorithm for 2D convex hull"""
    n = len(points)
    if n < 3:
        return points
    
    # Find leftmost point
    leftmost = min(range(n), key=lambda i: points[i][0])
    
    hull = []
    current = leftmost
    
    while True:
        hull.append(current)
        next_point = (current + 1) % n
        
        for i in range(n):
            if cross_product_2d(points[current], points[i], points[next_point]) > 0:
                next_point = i
        
        current = next_point
        if current == leftmost:
            break
    
    return [points[i] for i in hull]

print("✓ 2D convex hull loaded")

## 2. Mesh Simplification (Quadric Error Metric)

### Edge Collapse

Simplify mesh by collapsing edges iteratively.

### Quadric Error Metric (QEM)

For vertex $\mathbf{v}$ on planes $\pi_1, \pi_2, ..., \pi_k$:

$$
Q = \sum_{i=1}^k \mathbf{p}_i \mathbf{p}_i^T
$$

where $\mathbf{p}_i = [a, b, c, d]^T$ is plane equation $ax + by + cz + d = 0$

Error for vertex at position $\mathbf{v} = [x, y, z, 1]^T$:

$$
E(\mathbf{v}) = \mathbf{v}^T Q \mathbf{v}
$$

### Edge Collapse Cost

For edge $(\mathbf{v}_1, \mathbf{v}_2)$, new vertex position:

$$
\mathbf{v}_{\text{new}} = \arg\min_{\mathbf{v}} \mathbf{v}^T (Q_1 + Q_2) \mathbf{v}
$$

In [ ]:
class QuadricError:
    """Quadric error matrix for vertex"""
    def __init__(self):
        # 4x4 symmetric matrix stored as 10 values
        self.q = np.zeros(10)
    
    def add_plane(self, a: float, b: float, c: float, d: float):
        """Add plane ax + by + cz + d = 0"""
        # Add plane^T * plane to quadric
        self.q[0] += a * a
        self.q[1] += a * b
        self.q[2] += a * c
        self.q[3] += a * d
        self.q[4] += b * b
        self.q[5] += b * c
        self.q[6] += b * d
        self.q[7] += c * c
        self.q[8] += c * d
        self.q[9] += d * d
    
    def error(self, x: float, y: float, z: float) -> float:
        """Compute error for vertex at (x, y, z)"""
        return (self.q[0] * x * x + 2 * self.q[1] * x * y + 2 * self.q[2] * x * z + 2 * self.q[3] * x +
                self.q[4] * y * y + 2 * self.q[5] * y * z + 2 * self.q[6] * y +
                self.q[7] * z * z + 2 * self.q[8] * z +
                self.q[9])
    
    def __add__(self, other):
        """Combine quadrics"""
        result = QuadricError()
        result.q = self.q + other.q
        return result

def compute_triangle_plane(v0: Vec3, v1: Vec3, v2: Vec3) -> Tuple[float, float, float, float]:
    """Compute plane equation for triangle"""
    edge1 = v1 - v0
    edge2 = v2 - v0
    normal = edge1.cross(edge2).normalize()
    
    a, b, c = normal.x, normal.y, normal.z
    d = -(a * v0.x + b * v0.y + c * v0.z)
    
    return a, b, c, d

print("✓ QEM simplification loaded")

## 3. Subdivision Surfaces

### Catmull-Clark Subdivision

For quad meshes, each iteration:

1. **Face points**: Average of face vertices
$$
\mathbf{F} = \frac{1}{n} \sum_{i=1}^n \mathbf{v}_i
$$

2. **Edge points**: Average of edge vertices + adjacent face points
$$
\mathbf{E} = \frac{\mathbf{v}_1 + \mathbf{v}_2 + \mathbf{F}_1 + \mathbf{F}_2}{4}
$$

3. **Vertex points**: Weighted average
$$
\mathbf{V}_{\text{new}} = \frac{\mathbf{F} + 2\mathbf{E} + (n-3)\mathbf{V}}{n}
$$

where $n$ is vertex valence.

### Loop Subdivision (Triangles)

For triangle meshes:

**New edge vertex**:
$$
\mathbf{v}_{\text{new}} = \frac{3(\mathbf{v}_1 + \mathbf{v}_2) + \mathbf{v}_3 + \mathbf{v}_4}{8}
$$

**Updated vertex**:
$$
\mathbf{v}_{\text{new}} = (1 - n\beta)\mathbf{v} + \beta \sum_{i=1}^n \mathbf{v}_i
$$

where $\beta = \frac{1}{n}\left(\frac{5}{8} - \left(\frac{3}{8} + \frac{1}{4}\cos\frac{2\pi}{n}\right)^2\right)$

In [ ]:
def subdivide_triangle_simple(vertices, triangles):
    """Simple triangle subdivision (1-to-4 split)"""
    new_vertices = list(vertices)
    new_triangles = []
    edge_midpoints = {}
    
    def get_edge_midpoint(v0, v1):
        edge = tuple(sorted([v0, v1]))
        if edge not in edge_midpoints:
            p0 = vertices[v0]
            p1 = vertices[v1]
            mid = Vec3(
                (p0.x + p1.x) / 2,
                (p0.y + p1.y) / 2,
                (p0.z + p1.z) / 2
            )
            edge_midpoints[edge] = len(new_vertices)
            new_vertices.append(mid)
        return edge_midpoints[edge]
    
    for tri in triangles:
        v0, v1, v2 = tri.v0, tri.v1, tri.v2
        
        # Get edge midpoints
        m01 = get_edge_midpoint(v0, v1)
        m12 = get_edge_midpoint(v1, v2)
        m20 = get_edge_midpoint(v2, v0)
        
        # Create 4 new triangles
        new_triangles.append(Triangle(v0, m01, m20))
        new_triangles.append(Triangle(v1, m12, m01))
        new_triangles.append(Triangle(v2, m20, m12))
        new_triangles.append(Triangle(m01, m12, m20))
    
    return new_vertices, new_triangles

print("✓ Subdivision loaded")

## 4. Mesh Smoothing

### Laplacian Smoothing

Move each vertex toward average of neighbors:

$$
\mathbf{v}_i' = \mathbf{v}_i + \lambda \mathbf{L}(\mathbf{v}_i)
$$

where **Laplacian**:
$$
\mathbf{L}(\mathbf{v}_i) = \frac{1}{|N(i)|} \sum_{j \in N(i)} (\mathbf{v}_j - \mathbf{v}_i)
$$

$N(i)$ = neighbors of vertex $i$, $\lambda$ = smoothing factor (typically 0.5)

### Taubin Smoothing

Two-step process to avoid shrinkage:

1. Smoothing: $\lambda > 0$
2. Inflation: $\mu < 0$ where $\mu < -\lambda$

Acts as low-pass filter without volume loss.

In [ ]:
def build_adjacency(triangles, num_vertices):
    """Build vertex adjacency list"""
    adjacency = [set() for _ in range(num_vertices)]
    
    for tri in triangles:
        adjacency[tri.v0].add(tri.v1)
        adjacency[tri.v0].add(tri.v2)
        adjacency[tri.v1].add(tri.v0)
        adjacency[tri.v1].add(tri.v2)
        adjacency[tri.v2].add(tri.v0)
        adjacency[tri.v2].add(tri.v1)
    
    return adjacency

def laplacian_smooth(vertices, adjacency, lambda_factor=0.5):
    """Laplacian mesh smoothing"""
    new_vertices = []
    
    for i, v in enumerate(vertices):
        if len(adjacency[i]) == 0:
            new_vertices.append(v)
            continue
        
        # Compute Laplacian (average of neighbors - current)
        avg = Vec3(0, 0, 0)
        for neighbor_idx in adjacency[i]:
            avg = avg + vertices[neighbor_idx]
        avg = avg / len(adjacency[i])
        
        laplacian = avg - v
        new_pos = v + laplacian * lambda_factor
        new_vertices.append(new_pos)
    
    return new_vertices

def taubin_smooth(vertices, adjacency, lambda_val=0.5, mu_val=-0.53, iterations=10):
    """Taubin smoothing (shrinkage-free)"""
    result = vertices
    
    for _ in range(iterations):
        # Smoothing step
        result = laplacian_smooth(result, adjacency, lambda_val)
        # Inflation step
        result = laplacian_smooth(result, adjacency, mu_val)
    
    return result

print("✓ Mesh smoothing loaded")

## Example 1: 2D Convex Hull

In [ ]:
# Generate random 2D points
np.random.seed(42)
num_points = 30
points = [(np.random.uniform(-5, 5), np.random.uniform(-5, 5)) for _ in range(num_points)]

# Compute convex hull
hull = convex_hull_2d(points)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))

# Plot all points
all_x = [p[0] for p in points]
all_y = [p[1] for p in points]
ax.scatter(all_x, all_y, c='blue', s=50, alpha=0.6, label='Points')

# Plot hull
hull_x = [p[0] for p in hull] + [hull[0][0]]
hull_y = [p[1] for p in hull] + [hull[0][1]]
ax.plot(hull_x, hull_y, 'r-', linewidth=2, label='Convex Hull')
ax.scatter(hull_x[:-1], hull_y[:-1], c='red', s=100, marker='s', zorder=5)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('2D Convex Hull (Gift Wrapping)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

print(f"Input: {num_points} points")
print(f"Hull: {len(hull)} vertices")

## Example 2: Triangle Subdivision

In [ ]:
# Create simple tetrahedron
vertices = [
    Vec3(0, 1, 0),
    Vec3(-1, -0.5, -0.5),
    Vec3(1, -0.5, -0.5),
    Vec3(0, -0.5, 1)
]

triangles = [
    Triangle(0, 1, 2),
    Triangle(0, 2, 3),
    Triangle(0, 3, 1),
    Triangle(1, 3, 2)
]

# Subdivide twice
v1, t1 = subdivide_triangle_simple(vertices, triangles)
v2, t2 = subdivide_triangle_simple(v1, t1)

# Visualize
fig = plt.figure(figsize=(15, 5))

def plot_mesh(ax, verts, tris, title):
    for tri in tris:
        pts = np.array([
            verts[tri.v0].to_array(),
            verts[tri.v1].to_array(),
            verts[tri.v2].to_array(),
            verts[tri.v0].to_array()
        ])
        ax.plot3D(pts[:, 0], pts[:, 1], pts[:, 2], 'b-', linewidth=1)
    
    # Plot vertices
    vx = [v.x for v in verts]
    vy = [v.y for v in verts]
    vz = [v.z for v in verts]
    ax.scatter(vx, vy, vz, c='red', s=20)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(title)

ax1 = fig.add_subplot(131, projection='3d')
plot_mesh(ax1, vertices, triangles, f'Original\n{len(vertices)} vertices, {len(triangles)} triangles')

ax2 = fig.add_subplot(132, projection='3d')
plot_mesh(ax2, v1, t1, f'Subdivision 1\n{len(v1)} vertices, {len(t1)} triangles')

ax3 = fig.add_subplot(133, projection='3d')
plot_mesh(ax3, v2, t2, f'Subdivision 2\n{len(v2)} vertices, {len(t2)} triangles')

plt.tight_layout()
plt.show()

print("Subdivision increases mesh resolution for smooth surfaces.")

## Example 3: Mesh Smoothing

In [ ]:
# Create noisy mesh (sphere with noise)
def create_noisy_sphere(subdivisions=2, noise_level=0.1):
    # Start with icosahedron approximation
    t = (1.0 + math.sqrt(5.0)) / 2.0
    
    vertices = [
        Vec3(-1, t, 0).normalize(), Vec3(1, t, 0).normalize(),
        Vec3(-1, -t, 0).normalize(), Vec3(1, -t, 0).normalize(),
        Vec3(0, -1, t).normalize(), Vec3(0, 1, t).normalize(),
        Vec3(0, -1, -t).normalize(), Vec3(0, 1, -t).normalize(),
        Vec3(t, 0, -1).normalize(), Vec3(t, 0, 1).normalize(),
        Vec3(-t, 0, -1).normalize(), Vec3(-t, 0, 1).normalize()
    ]
    
    triangles = [
        Triangle(0, 11, 5), Triangle(0, 5, 1), Triangle(0, 1, 7), Triangle(0, 7, 10), Triangle(0, 10, 11),
        Triangle(1, 5, 9), Triangle(5, 11, 4), Triangle(11, 10, 2), Triangle(10, 7, 6), Triangle(7, 1, 8),
        Triangle(3, 9, 4), Triangle(3, 4, 2), Triangle(3, 2, 6), Triangle(3, 6, 8), Triangle(3, 8, 9),
        Triangle(4, 9, 5), Triangle(2, 4, 11), Triangle(6, 2, 10), Triangle(8, 6, 7), Triangle(9, 8, 1)
    ]
    
    # Subdivide for more vertices
    for _ in range(subdivisions):
        vertices, triangles = subdivide_triangle_simple(vertices, triangles)
        # Normalize to sphere
        vertices = [v.normalize() for v in vertices]
    
    # Add noise
    noisy_vertices = []
    for v in vertices:
        noise = np.random.uniform(-noise_level, noise_level)
        noisy_v = v * (1.0 + noise)
        noisy_vertices.append(noisy_v)
    
    return noisy_vertices, triangles

# Create noisy sphere
noisy_verts, tris = create_noisy_sphere(subdivisions=1, noise_level=0.15)

# Build adjacency
adj = build_adjacency(tris, len(noisy_verts))

# Smooth
smoothed_verts = taubin_smooth(noisy_verts, adj, iterations=5)

# Visualize
fig = plt.figure(figsize=(12, 5))

ax1 = fig.add_subplot(121, projection='3d')
plot_mesh(ax1, noisy_verts, tris, 'Noisy Mesh')

ax2 = fig.add_subplot(122, projection='3d')
plot_mesh(ax2, smoothed_verts, tris, 'Taubin Smoothed')

plt.tight_layout()
plt.show()

print("Taubin smoothing removes noise while preserving volume.")

## Summary

**Geometric algorithms** are essential for mesh processing:

### Key Concepts

1. **Convex Hull**
   - Gift wrapping: O(nh) time
   - Graham scan: O(n log n) time
   - Applications: collision detection, bounding volumes

2. **Mesh Simplification**
   - Edge collapse with QEM
   - Preserves visual appearance
   - LOD (Level of Detail) generation
   - Reduces polygon count for performance

3. **Subdivision Surfaces**
   - Catmull-Clark (quads) → smooth limit surface
   - Loop (triangles) → smooth triangulation
   - 1-to-4 split per iteration
   - Creates smooth organic shapes

4. **Mesh Smoothing**
   - Laplacian: simple but causes shrinkage
   - Taubin: two-step, volume preserving
   - Removes high-frequency noise
   - Iterative local averaging

5. **Advanced Topics**
   - Mesh parameterization (UV unwrapping)
   - Boolean operations (CSG)
   - Mesh repair (hole filling)
   - Remeshing (uniform triangle size)

### Applications

- **CAD/CAM**: Modeling and manufacturing
- **Animation**: Character modeling with subdivision
- **Games**: LOD generation for performance
- **3D Printing**: Mesh repair and optimization
- **Simulation**: Clean meshes for FEM/physics

Geometric algorithms are fundamental tools in the 3D graphics pipeline for creating, optimizing, and processing mesh geometry.